In [49]:
import pandas as pd
import numpy as np
from scipy.special import expit
from functools import reduce
from online_logistic_regression import OnlineLogisticRegression


In [3]:
from environment import ContextualEnvironment
from policies import KLUCBSegmentPolicy, RandomPolicy, ExploreThenCommitSegmentPolicy, EpsilonGreedySegmentPolicy, TSSegmentPolicy, LinearTSPolicy
import argparse
import json
import logging
import numpy as np
import pandas as pd
import time

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument("--users_path", type = str, default = "data/user_features.csv", required = False,
                    help = "Path to user features file")
parser.add_argument("--playlists_path", type = str, default = "data/playlist_features.csv", required = False,
                    help = "Path to playlist features file")
parser.add_argument("--output_path", type = str, default = "results.json", required = False,
                    help = "Path to json file to save regret values")
parser.add_argument("--policies", type = str, default = "random,ts-seg-naive", required = False,
                    help = "Bandit algorithms to evaluate, separated by commas")
parser.add_argument("--n_recos", type = int, default = 12, required = False,
                    help = "Number of slots L in the carousel i.e. number of recommendations to provide")
parser.add_argument("--l_init", type = int, default = 3, required = False,
                    help = "Number of slots L_init initially visible in the carousel")
parser.add_argument("--n_users_per_round", type = int, default = 20000, required = False,
                    help = "Number of users randomly selected (with replacement) per round")
parser.add_argument("--n_rounds", type = int, default = 100, required = False,
                    help = "Number of simulated rounds")
parser.add_argument("--print_every", type = int, default = 10, required = False,
                    help = "Print cumulative regrets every 'print_every' round")

args = parser.parse_args(args = [])

In [5]:
playlists_df = pd.read_csv('data/playlist_features.csv')

users_df = pd.read_csv('data/user_features_small.csv')

n_users = len(users_df)
n_playlists = len(playlists_df)

In [6]:
user_features = np.array(users_df.drop(["segment"], axis = 1))
user_features = np.concatenate([user_features, np.ones((n_users,1))], axis = 1)
playlist_features = np.array(playlists_df)

In [7]:
user_segment = np.array(users_df.segment)

In [8]:
user_segment

array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [9]:
def set_policies(policies_name, user_segment, user_features, n_playlists):
    # Please see section 3.3 of RecSys paper for a description of policies
    POLICIES_SETTINGS = {
        'random' : RandomPolicy(n_playlists),
        'etc-seg-explore' : ExploreThenCommitSegmentPolicy(user_segment, n_playlists, min_n = 100, cascade_model = True),
        'etc-seg-exploit' : ExploreThenCommitSegmentPolicy(user_segment, n_playlists, min_n = 20, cascade_model = True),
        'epsilon-greedy-explore' : EpsilonGreedySegmentPolicy(user_segment, n_playlists, epsilon = 0.1, cascade_model = True),
        'epsilon-greedy-exploit' : EpsilonGreedySegmentPolicy(user_segment, n_playlists, epsilon = 0.01, cascade_model = True),
        'kl-ucb-seg' : KLUCBSegmentPolicy(user_segment, n_playlists, cascade_model = True),
        'ts-seg-naive' : TSSegmentPolicy(user_segment, n_playlists, alpha_zero = 1, beta_zero = 1, cascade_model = True),
        'ts-seg-pessimistic' : TSSegmentPolicy(user_segment, n_playlists, alpha_zero = 1, beta_zero = 99, cascade_model = True),
        'ts-lin-naive' : LinearTSPolicy(user_features, n_playlists, bias = 0.0, cascade_model = True),
        'ts-lin-pessimistic' : LinearTSPolicy(user_features, n_playlists, bias = -5.0, cascade_model = True),
        # Versions of epsilon-greedy-explore and ts-seg-pessimistic WITHOUT cascade model
        'epsilon-greedy-explore-no-cascade' : EpsilonGreedySegmentPolicy(user_segment, n_playlists, epsilon = 0.1, cascade_model = False),
        'ts-seg-pessimistic-no-cascade' : TSSegmentPolicy(user_segment, n_playlists, alpha_zero = 1, beta_zero = 99, cascade_model = False)
    }

    return [POLICIES_SETTINGS[name] for name in policies_name]

In [10]:
po = 'random,ts-lin-pessimistic,ts-seg-pessimistic'
policies_name = po.split(",")

In [12]:
policies = set_policies(policies_name, user_segment, user_features, n_playlists)

init ========================================
init ========================================


In [13]:
policies_name

['random', 'ts-lin-pessimistic', 'ts-seg-pessimistic']

In [14]:
n_policies = len(policies) # 3
n_users_per_round = args.n_users_per_round
n_rounds = args.n_rounds
overall_rewards = np.zeros((n_policies, n_rounds))
overall_optimal_reward = np.zeros(n_rounds)

In [15]:
cont_env = ContextualEnvironment(user_features, playlist_features, user_segment, args.n_recos)

In [16]:
cont_env.th_rewards

array([0.51069545, 0.60661954, 0.45462272, 0.35597582, 0.45974146,
       0.33826388, 0.94396647, 0.6756706 , 0.35810638])

In [17]:
user_ids = np.random.choice(range(n_users), n_users_per_round) # range * 수

In [18]:
np.take(cont_env.th_rewards, user_ids).sum()

10452.071280730333

In [19]:
n_users = 9
u = 0
step = 100000

In [20]:
probas = np.take(user_features, range(0,9), axis = 0).dot(playlist_features.T)

In [21]:
so = np.argsort(-probas)[:, :12]

In [22]:
so1 = np.take(playlist_features, so, axis = 0)

In [24]:
batch_user_ids = range(0,9)
batch_recos = so
batch_user_features = np.take(user_features, batch_user_ids, axis = 0)              # batch 안에 있는 user에 대한 user_feature만 추출
batch_playlist_features = np.take(playlist_features, batch_recos, axis = 0)    # 추천리스트에 있는 item에 대한 feature 추출
n_users = len(batch_user_ids)                                                       # batch 내에 있는 유저 수
th_reward = np.zeros(n_users)                                                       # zero vector 생성 (user 수)
for i in range(n_users):                                                            # user 수 만큼 for 문 수행
    probas = expit(batch_user_features[i].dot(batch_playlist_features[i].T)) 
    th_reward[i] = 1 - reduce(lambda x,y : x * y, 1 - probas)

In [27]:
batch_user_features[i].dot(batch_playlist_features[i].T)

array([-2.66080412, -2.93343552, -2.95593961, -3.30119431, -3.36031116,
       -3.42852429, -3.46619029, -3.47532706, -3.49337995, -3.56729809,
       -3.62311179, -3.65233738])

In [28]:
expit(batch_user_features[i].dot(batch_playlist_features[i].T)) 

array([0.06532622, 0.05052526, 0.04945654, 0.03553024, 0.03355913,
       0.03141581, 0.03028968, 0.03002246, 0.02950118, 0.02745687,
       0.02600514, 0.02527506])

In [29]:
for i in range(9):                                                            # user 수 만큼 for 문 수행
    probas = expit(np.take(user_features, range(0,9), axis = 0)[i].dot(so1[i].T))        # sigmoid 
    th_reward[i] = 1 - reduce(lambda x,y : x * y, 1 - probas)

In [30]:
th_rewards = np.zeros(user_features.shape[0]) 

In [31]:
th_rewards[0:9] = th_reward

In [32]:
user_segment = np.array(users_df.segment)

In [33]:
user_segment

array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [34]:
n_segments = len(np.unique(user_segment))                              # unique 한 segment 수
segment_recos = np.zeros((n_segments, 12), dtype = np.int64)                 # (segment 수, 추천리스트 수) > segment 별로 동일한 추천리스트 생성
for i in range(n_segments):
    mean_probas = np.mean(expit(np.take(user_features, np.where(user_segment == i)[0], axis = 0).dot(playlist_features.T)), axis = 0)
    reward = 1 - reduce(lambda x,y : x * y, 1 + np.sort(-mean_probas)[:12])
    segment_recos[i] = np.argsort(-mean_probas)[:12]

In [35]:
segment_recos

array([[195, 371,  51, 408, 128, 413, 251, 718, 396, 172, 178, 252]],
      dtype=int64)

In [38]:
users_ids = range(u, min(n_users, u + step))

In [40]:
user_segment = np.take(user_segment, users_ids)                    # batch 안에 있는 user의 segment 추출
opt_recos = np.take(segment_recos, user_segment, axis = 0)

In [43]:
th_rewards

array([0.51069545, 0.60661954, 0.45462272, 0.35597582, 0.45974146,
       0.33826388, 0.94396647, 0.6756706 , 0.35810638])

In [44]:
user_ids = np.random.choice(range(n_users), n_users_per_round)                                  # 전체 유저에서 batch 크기 만큼 샘플링 / 중복 유저도 가능한데...?
overall_optimal_reward = np.take(cont_env.th_rewards, user_ids).sum()

In [45]:
user_ids

array([3, 0, 4, ..., 1, 4, 5])

In [ ]:
for j in range(n_policies):
    # Compute n_recos recommendations
    recos = policies[j].recommend_to_users_batch(user_ids, args.n_recos, args.l_init)
    # Compute rewards
    rewards = cont_env.simulate_batch_users_reward(batch_user_ids= user_ids, batch_recos=recos)
    # Update policy based on rewards
    policies[j].update_policy(user_ids, recos, rewards, args.l_init)
    overall_rewards[j,i] = rewards.sum()
# Print info
if i == 0 or (i+1) % print_every == 0 or i+1 == n_rounds:
    logger.info("Round: %d/%d. Elapsed time: %f sec." % (i+1, n_rounds, time.time() - start_time))
    logger.info("Cumulative regrets: \n%s \n" % "\n".join(["	%s : %s" % (policies_name[j], str(np.sum(overall_optimal_reward - overall_rewards[j]))) for j in range(n_policies)]))



In [136]:
policies[0].recommend_to_users_batch(user_ids, args.n_recos, args.l_init).shape

(20000, 12)

In [61]:
def __init__(self, user_features, n_playlists, bias=0.0, cascade_model=True):                       # LinearTSPolicy(user_features, n_playlists, bias = -5.0, cascade_model = True)
    self.user_features = user_features
    n_dim = user_features.shape[1]                                                                  # user feature dimension : 97
    self.n_playlists = n_playlists                                                                  # item 수 : 862
    self.models = [OnlineLogisticRegression(1, 1, n_dim, bias, 15) for i in range(n_playlists)]     # item
    self.m = np.zeros((n_playlists, n_dim))                                                         # item feature matrix
    self.m[:, -1] = bias
    self.q = np.ones((n_playlists, n_dim))
    self.n_dim = n_dim
    self.cascade_model = cascade_model

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])